In [ ]:
import numpy as np
import pandas as pd
import lightningchart as lc

lc.set_license('my-license-key')

data = pd.read_csv('../Dataset/mixed_categorical_numerical_data.csv')

# Histogram

In [14]:
x6_values = data['X6']
x7_values = data['X7']
x8_values = data['X8']

# Calculate histograms with logarithmic bins for X6, X7, and X8
bins_x6 = np.logspace(np.log10(x6_values.min() + 1e-6), np.log10(x6_values.max()), 30)
bins_x7 = np.linspace(x7_values.min(), x7_values.max(), 30)
bins_x8 = np.linspace(x8_values.min(), x8_values.max(), 30)

counts_x6, bin_edges_x6 = np.histogram(x6_values, bins=bins_x6)
counts_x7, bin_edges_x7 = np.histogram(x7_values, bins=bins_x7)
counts_x8, bin_edges_x8 = np.histogram(x8_values, bins=bins_x8)

# Prepare bar data for the histograms
bar_data_x6 = [
    {"category": f"{bin_edges_x6[i]:.2f}–{bin_edges_x6[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x6)
]

bar_data_x7 = [
    {"category": f"{bin_edges_x7[i]:.2f}–{bin_edges_x7[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x7)
]

bar_data_x8 = [
    {"category": f"{bin_edges_x8[i]:.2f}–{bin_edges_x8[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x8)
]

# Create a dashboard
dashboard = lc.Dashboard(columns=3, rows=1, theme=lc.Themes.Dark)

# Create BarCharts for X6, X7, and X8
chart_x6 = dashboard.BarChart(
    vertical=True,
    column_index=0,
    row_index=0
).set_title("Histogram of Grinding Thickness (X6) - Logarithmic Binning")

chart_x6.set_data(bar_data_x6)
chart_x6.set_sorting('disabled')
chart_x6.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},  # Low value
        {'value': 0.5, 'color': lc.Color('yellow')},  # Mid value
        {'value': 1, 'color': lc.Color('red')}  # High value
    ],
    percentage_values=True
)

chart_x7 = dashboard.BarChart(
    vertical=True,
    column_index=1,
    row_index=0
).set_title("Histogram of Number of Wires (X7)")
chart_x7.set_data(bar_data_x7)
chart_x7.set_sorting('disabled')
chart_x7.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},
        {'value': 0.5, 'color': lc.Color('yellow')},
        {'value': 1, 'color': lc.Color('red')}
    ],
    percentage_values=True
)

chart_x8 = dashboard.BarChart(
    vertical=True,
    column_index=2,
    row_index=0
).set_title("Histogram of Wire Width (X8)")
chart_x8.set_data(bar_data_x8)
chart_x8.set_sorting('disabled')
chart_x8.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},
        {'value': 0.5, 'color': lc.Color('yellow')},
        {'value': 1, 'color': lc.Color('red')}
    ],
    percentage_values=True
)

dashboard.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:35] "GET / HTTP/1.1" 200 -


# Box Plot

In [15]:
import pandas as pd
import numpy as np
import lightningchart as lc

# Create a LightningChart Box Plot
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Throughput Rate by Recipe Type (X5)'
)

chart.set_series_background_color(lc.Color(0, 255, 255))
chart.get_default_y_axis().set_title("Throughput Rate (Y)")
chart.get_default_x_axis().set_title("Recipe Type (X5)")

# Process the data to create box plot statistics for each category in X5
categories = data['X5'].unique()  # Get unique recipe types
dataset = []
x_values_outlier = []
y_values_outlier = []

for i, category in enumerate(categories):
    category_data = data[data['X5'] == category]['Y'].dropna().tolist()  # Filter Y values for the category
    start = i + 1
    end = start + 0.5
    
    lowerQuartile = float(np.percentile(category_data, 25))
    upperQuartile = float(np.percentile(category_data, 75))
    median = float(np.median(category_data))
    iqr = upperQuartile - lowerQuartile
    lowerExtreme = max(lowerQuartile - 1.5 * iqr, np.min(category_data))  # Whisker lower bound
    upperExtreme = min(upperQuartile + 1.5 * iqr, np.max(category_data))  # Whisker upper bound

    
    dataset.append({
        'start': start,
        'end': end,
        'lowerQuartile': lowerQuartile,
        'upperQuartile': upperQuartile,
        'median': median,
        'lowerExtreme': lowerExtreme,
        'upperExtreme': upperExtreme
    })
    
    # Identify outliers
    iqr = upperQuartile - lowerQuartile
    lower_bound = lowerQuartile - 1.5 * iqr
    upper_bound = upperQuartile + 1.5 * iqr
    outliers = [y for y in category_data if y < lower_bound or y > upper_bound]
    
    for outlier in outliers:
        x_values_outlier.append(start + 0.25)  # Center the outliers in the box
        y_values_outlier.append(outlier)

# Add box plot data to the chart
box_series = chart.add_box_series()
box_series.add_multiple(dataset)

# Add outliers as red points
outlier_series = chart.add_point_series(sizes=True, lookup_values=True)
outlier_series.set_point_color(lc.Color('red'))
outlier_series.append_samples(
    x_values=x_values_outlier,
    y_values=y_values_outlier,
    sizes=[10] * len(y_values_outlier)
)

chart.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:35] "GET / HTTP/1.1" 200 -


# Heatmap for Numerical Features

In [16]:
import lightningchart as lc
import pandas as pd
import numpy as np

# Select only numeric columns
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Calculate correlation for numeric columns
correlation_matrix = numeric_data.corr()

# Convert correlation matrix to a numpy array
correlation_array = correlation_matrix.to_numpy()

# Extract column names for labeling
labels = correlation_matrix.columns

# Initialize LightningChart
chart = lc.ChartXY(
    title="Correlation Heatmap for Numerical Features",
    theme=lc.Themes.Dark
)

# Set up the heatmap grid
grid_size_x, grid_size_y = correlation_array.shape
heatmap_series = chart.add_heatmap_grid_series(
    columns=grid_size_x,
    rows=grid_size_y,
)

# Configure heatmap properties
heatmap_series.set_start(x=0, y=0)
heatmap_series.set_end(x=grid_size_x, y=grid_size_y)
heatmap_series.set_step(x=1, y=1)
heatmap_series.set_wireframe_stroke(thickness=1, color=lc.Color('white'))
heatmap_series.invalidate_intensity_values(correlation_array.tolist())
heatmap_series.set_intensity_interpolation(False)

# Define color palette for correlation
palette_steps = [
    {"value": -1, "color": lc.Color('blue')},     # Strong negative correlation
    {"value": 0, "color": lc.Color('white')},     # No correlation
    {"value": 1, "color": lc.Color('red')}        # Strong positive correlation
]
heatmap_series.set_palette_coloring(
    steps=palette_steps,
    look_up_property='value',
    interpolate=True
)

# Set up axes
x_axis = chart.get_default_x_axis()
y_axis = chart.get_default_y_axis()
x_axis.set_tick_strategy('Empty')
y_axis.set_tick_strategy('Empty')

# Add labels for both axes
for i, label in enumerate(labels):
    custom_tick_x = x_axis.add_custom_tick().set_tick_label_rotation(90)
    custom_tick_x.set_value(i + 0.5)  # Position labels between grid lines
    custom_tick_x.set_text(label)

    custom_tick_y = y_axis.add_custom_tick()
    custom_tick_y.set_value(i + 0.5)
    custom_tick_y.set_text(label)

# Add legend
chart.add_legend(data=heatmap_series).set_margin(-20).set_dragging_mode("draggable")

chart.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:36] "GET / HTTP/1.1" 200 -


# Pair Plot

In [17]:
import numpy as np
import pandas as pd
import lightningchart as lc
from scipy.stats import gaussian_kde
# Select the features for the pair plot
features = ['X6', 'X7', 'X8', 'Y']

# Initialize LightningChart Dashboard
dashboard = lc.Dashboard(
    rows=len(features),
    columns=len(features),
    theme=lc.Themes.Light
)

# Helper function to create density plots (diagonal)
def create_density_chart(dashboard, title, values, feature, column_index, row_index):
    chart = dashboard.ChartXY(
        column_index=column_index,
        row_index=row_index
    )
    chart.set_title(title)
    chart.set_padding(0)

    # Compute density
    values_np = np.array(values)
    density = gaussian_kde(values_np)
    x_vals = np.linspace(values_np.min(), values_np.max(), 100)
    y_vals = density(x_vals)

    # Add area series
    series = chart.add_area_series()
    series.add(x_vals.tolist(), y_vals.tolist())
    series.set_fill_color(lc.Color('blue'))
    series.set_name(f'Density - {feature}')

    # Set axis titles
    chart.get_default_x_axis().set_title('Value')
    chart.get_default_y_axis().set_title('Density')

# Helper function to create scatter plots
def create_scatter_chart(dashboard, title, x_values, y_values, x_feature, y_feature, column_index, row_index):
    chart = dashboard.ChartXY(
        column_index=column_index,
        row_index=row_index
    )
    chart.set_title(title)
    chart.set_padding(0)

    # Add scatter points
    scatter_series = chart.add_point_series()
    scatter_series.add(x_values, y_values)

    # Apply color map to scatter points
    scatter_series.set_palette_point_coloring(
        steps=[
            {"value": min(y_values), "color": lc.Color('blue')},
            {"value": (max(y_values) + min(y_values)) / 2, "color": lc.Color('green')},
            {"value": max(y_values), "color": lc.Color('red')}
        ],
        look_up_property='y',  # Map colors based on Y values
        interpolate=True  # Smooth interpolation between colors
    )

    scatter_series.set_point_size(3)

    # Set axis titles
    chart.get_default_x_axis().set_title(x_feature)
    chart.get_default_y_axis().set_title(y_feature)

# Generate the pair plot
for row_index, y_col in enumerate(features):
    for column_index, x_col in enumerate(features):
        if row_index == column_index:
            # Diagonal: Density Plot
            values = data[y_col].dropna().astype(float).tolist()
            create_density_chart(dashboard, f'Density of {x_col}', values, x_col, column_index, row_index)
        else:
            # Off-Diagonal: Scatter Plot
            x_values = data[x_col].dropna().astype(float).tolist()
            y_values = data[y_col].dropna().astype(float).tolist()
            create_scatter_chart(dashboard, f'{x_col} vs {y_col}', x_values, y_values, x_col, y_col, column_index, row_index)

dashboard.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:36] "GET / HTTP/1.1" 200 -


# Rectangle Series

In [18]:
import numpy as np
import pandas as pd
import lightningchart as lc

x6_values = data['X6']
x7_values = data['X7']
x8_values = data['X8']

# Calculate histograms for X6, X7, and X8
bins_x6 = np.linspace(x6_values.min(), x6_values.max(), 30)
bins_x7 = np.linspace(x7_values.min(), x7_values.max(), 30)
bins_x8 = np.linspace(x8_values.min(), x8_values.max(), 30)

counts_x6, bin_edges_x6 = np.histogram(x6_values, bins=bins_x6)
counts_x7, bin_edges_x7 = np.histogram(x7_values, bins=bins_x7)
counts_x8, bin_edges_x8 = np.histogram(x8_values, bins=bins_x8)

# Convert counts to Python integers for compatibility
counts_x6 = counts_x6.astype(int).tolist()
counts_x7 = counts_x7.astype(int).tolist()
counts_x8 = counts_x8.astype(int).tolist()

# Function to create step charts using thin strokes
def add_step_chart(dashboard, column_index, row_index, bins, counts, title, x_label):
    chart = dashboard.ChartXY(
        title=title,
        column_index=column_index,
        row_index=row_index,
    )
    rectangle_series = chart.add_rectangle_series()

    # Add rectangles for step-like visualization
    for i in range(len(counts)):
        if counts[i] > 0:
            x1 = float(bins[i])
            x2 = float(bins[i + 1])
            y = float(counts[i])  # Use raw counts

            # Use thin rectangles to mimic step lines
            rect = rectangle_series.add(x1, y - 0.5, x2, y + 0.5)  # Thin height
            rect.set_color(lc.Color(255, 255, 0, 255))  # Yellow for strokes
            rect.set_stroke(4, lc.Color('yellow'))  # Black border

    chart.get_default_x_axis().set_title(x_label)
    chart.get_default_y_axis().set_title("Count")
    chart.get_default_y_axis().set_interval(0, max(counts) + 10)

    return chart


# Create the dashboard
dashboard = lc.Dashboard(columns=3, rows=1, theme=lc.Themes.Dark)

# Add step charts for X6, X7, and X8
add_step_chart(
    dashboard,
    column_index=0,
    row_index=0,
    bins=bin_edges_x6,
    counts=counts_x6,
    title="Distribution of Grinding Thickness (X6)",
    x_label="Grinding Thickness (X6)"
)

add_step_chart(
    dashboard,
    column_index=1,
    row_index=0,
    bins=bin_edges_x7,
    counts=counts_x7,
    title="Distribution of Number of Wires (X7)",
    x_label="Number of Wires (X7)"
)

add_step_chart(
    dashboard,
    column_index=2,
    row_index=0,
    bins=bin_edges_x8,
    counts=counts_x8,
    title="Distribution of Wire Width (X8)",
    x_label="Wire Width (X8)"
)

dashboard.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:37] "GET / HTTP/1.1" 200 -


# Stacked Area Chart

In [19]:
import pandas as pd
import lightningchart as lc

# Group data by Machine Type (X1) and calculate cumulative throughput rate
data['Cumulative_Y'] = data.groupby('X1')['Y'].cumsum()

# Group data by Product Type (X2) and calculate cumulative throughput rate
data['Cumulative_Y_Product'] = data.groupby('X2')['Y'].cumsum()

# Create a LightningChart Dashboard
dashboard = lc.Dashboard(rows=2, columns=1, theme=lc.Themes.Dark)

# Function to add area chart
def add_area_chart(dashboard, row_index, title, x_data, y_data_groups, x_label, y_label, legend_labels):
    chart = dashboard.ChartXY(row_index=row_index, column_index=0, title=title)
    chart.get_default_x_axis().set_title(x_label)
    chart.get_default_y_axis().set_title(y_label)
    legend = chart.add_legend()

    for group_label, group_data in y_data_groups.items():
        area_series = chart.add_area_series()
        area_series.set_name(legend_labels[group_label])
        area_series.add(x_data, group_data)
        legend.add(area_series)

    return chart

# Prepare data for Machine Type (X1) cumulative chart
machine_types = data['X1'].unique()
machine_data_groups = {
    machine: data[data['X1'] == machine]['Cumulative_Y'].values
    for machine in machine_types
}
observation_order = data.index
machine_labels = {machine: f"Machine Type: {machine}" for machine in machine_types}

# Add Machine Type chart
add_area_chart(
    dashboard,
    row_index=0,
    title="Cumulative Throughput Rate by Machine Type",
    x_data=observation_order,
    y_data_groups=machine_data_groups,
    x_label="Observation Order",
    y_label="Cumulative Throughput Rate",
    legend_labels=machine_labels
)

# Prepare data for Product Type (X2) cumulative chart
product_types = data['X2'].unique()
product_data_groups = {
    product: data[data['X2'] == product]['Cumulative_Y_Product'].values
    for product in product_types
}
product_labels = {product: f"Product Type: {product}" for product in product_types}

# Add Product Type chart
add_area_chart(
    dashboard,
    row_index=1,
    title="Cumulative Throughput Rate by Product Type",
    x_data=observation_order,
    y_data_groups=product_data_groups,
    x_label="Observation Order",
    y_label="Cumulative Throughput Rate",
    legend_labels=product_labels
)

dashboard.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:37] "GET / HTTP/1.1" 200 -


# Regression

In [20]:
import pandas as pd
import numpy as np
import lightningchart as lc
from sklearn.linear_model import LinearRegression

# Extract X6, X7, and Y
x6 = data['X6'].values.astype(float).reshape(-1, 1)
x7 = data['X7'].values.astype(float).reshape(-1, 1)
y = data['Y'].values.astype(float)

# Fit regression models for X6 and X7
reg_x6 = LinearRegression().fit(x6, y)
reg_x7 = LinearRegression().fit(x7, y)

# Predict values for the regression line
x6_range = np.linspace(x6.min(), x6.max(), 100).reshape(-1, 1)
x7_range = np.linspace(x7.min(), x7.max(), 100).reshape(-1, 1)
y_x6_pred = reg_x6.predict(x6_range)
y_x7_pred = reg_x7.predict(x7_range)

# Create a dashboard
dash = lc.Dashboard(rows=1, columns=2, theme=lc.Themes.Light)

# Chart for X6 vs. Y
chart_x6 = dash.ChartXY(row_index=0, column_index=0, title="Regression: X6 vs. Y")
chart_x6.get_default_x_axis().set_title("Grinding Thickness (X6)")
chart_x6.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X6 vs. Y
scatter_x6 = chart_x6.add_point_series()
scatter_x6.set_point_color(lc.Color(0, 0, 255, 100))  # Semi-transparent blue
for x, y_val in zip(x6, y):
    scatter_x6.append_samples(x_values=float(x[0]), y_values=float(y_val))

# Proper regression line for X6 vs. Y
line_x6 = chart_x6.add_line_series()
line_x6.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
for x, y_val in zip(x6_range, y_x6_pred):
    line_x6.append_samples(x_values=float(x[0]), y_values=float(y_val))

scatter_x6.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 255, 0)},  # Green for low values
        {"value": y.max(), "color": lc.Color(0, 0, 255)},  # Blue for high values
    ],
    look_up_property="y",
    interpolate=True
)

# Chart for X7 vs. Y
chart_x7 = dash.ChartXY(row_index=0, column_index=1, title="Regression: X7 vs. Y")
chart_x7.get_default_x_axis().set_title("Feature (X7)")
chart_x7.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X7 vs. Y
scatter_x7 = chart_x7.add_point_series()
scatter_x7.set_point_color(lc.Color(0, 255, 0, 100))  # Semi-transparent green
for x, y_val in zip(x7, y):
    scatter_x7.append_samples(x_values=float(x[0]), y_values=float(y_val))

# Proper regression line for X7 vs. Y
line_x7 = chart_x7.add_line_series()
line_x7.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
for x, y_val in zip(x7_range, y_x7_pred):
    line_x7.append_samples(x_values=float(x[0]), y_values=float(y_val))

scatter_x7.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 255, 0)},  # Green for low values
        {"value": y.max(), "color": lc.Color(0, 0, 255)},  # Blue for high values
    ],
    look_up_property="y",
    interpolate=True
)

dash.open(method='browser')


127.0.0.1 - - [24/Feb/2025 16:59:39] "GET / HTTP/1.1" 200 -


# t-SNE Visualization

In [21]:
import pandas as pd
from sklearn.manifold import TSNE
import lightningchart as lc

# Perform t-SNE on numerical features
numerical_data = data.drop(columns=['X1', 'X2', 'X3', 'X4', 'X5', 'Y'])
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_result = tsne.fit_transform(numerical_data)

# Add t-SNE results to the dataframe
data['t-SNE1'] = tsne_result[:, 0].astype(float)  # Ensure float type
data['t-SNE2'] = tsne_result[:, 1].astype(float)  # Ensure float type

# Map Machine Type (X1) to numerical values (for palette lookup)
data['MachineTypeCode'] = data['X1'].astype('category').cat.codes.astype(int)  # Ensure int type

# Create LightningChart Scatter Plot
chart = lc.ChartXY(title='t-SNE Visualization of Machine Type (X1)', theme=lc.Themes.Light)

point_series = chart.add_point_series(colors=True, lookup_values=True)

# Add t-SNE data to the point series
for _, row in data.iterrows():
    point_series.append_sample(
        x=float(row['t-SNE1']), 
        y=float(row['t-SNE2']), 
        lookup_value=int(row['MachineTypeCode']) 
    )

# Define a palette
point_series.set_palette_point_coloring(
    steps=[
        {'value': int(data['MachineTypeCode'].min()), 'color': lc.Color('purple')},  # Blue
        {'value': int(data['MachineTypeCode'].max()), 'color': lc.Color('yellow')}   # Red 
    ],
    look_up_property='value',
    interpolate=True  
)

chart.get_default_x_axis().set_title('t-SNE Component 1')
chart.get_default_y_axis().set_title('t-SNE Component 2')

chart.open(method='browser')

c:\Users\Soroush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
127.0.0.1 - - [24/Feb/2025 17:00:07] "GET / HTTP/1.1" 200 -


# Streaming

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor  # Import XGBoost
import lightningchart as lc
import time

# Separate the target variable (Y) and features (X)
target = data["Y"]
features = data.drop(columns=["Y"])

# Identify categorical and numerical columns
categorical_columns = ["X1", "X2", "X3", "X4", "X5"]
numerical_columns = [col for col in features.columns if col not in categorical_columns]

# One-hot encode the categorical variables
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_categorical = encoder.fit_transform(features[categorical_columns])

# Combine encoded categorical variables with numerical ones
encoded_features = np.hstack((encoded_categorical, features[numerical_columns].values))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    encoded_features, target, test_size=0.2, random_state=42
)

# Train an XGBoost regression model
model = XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

# Predict and calculate residuals on the test set
y_pred = model.predict(X_test)
residuals = np.abs(y_test - y_pred)

# Shift y-values to make all values positive
shift_value = abs(min(y_test.min(), y_pred.min(), residuals.min()))
y_test_shifted = y_test + shift_value
y_pred_shifted = y_pred + shift_value
residuals_shifted = residuals + shift_value

# Normalize residuals to derive certainty values (1 - normalized residuals)
certainty = 1 - (residuals_shifted / residuals_shifted.max())
certainty_values = certainty.values  # Ensure we have values for visualization

# Parameters for heatmap visualization
num_rows = 100
time_delay = 0.2
batch_size = 10  # Steps to process per batch
line_chart_delay = 30  # 2-second delay for line chart

# Determine the full range of y-axis values (spanning actual and predicted)
y_min = min(y_test_shifted.min(), y_pred_shifted.min())
y_max = max(y_test_shifted.max(), y_pred_shifted.max())
y_range = np.linspace(y_min, y_max, num_rows)


# Generate heatmap matrix based on prediction range
def generate_certainty_heatmap(y_pred_value, certainty_value):
    column = 1 - np.abs(y_range - y_pred_value) / (
        y_max - y_min
    )  # Intensity based on proximity
    column = column * certainty_value  # Scale by certainty
    # column = np.clip(column, 0, 1)  # Normalize to [0, 1]
    return column


# Initialize heatmap matrix
columns = len(y_pred_shifted)
mat = np.zeros((num_rows, columns))

dashboard = lc.Dashboard(rows=2, columns=1, theme=lc.Themes.Dark)

# ---- First Row: Real y_pred and y_test Visualization ----
chart1 = dashboard.ChartXY(row_index=0, column_index=0).set_title(
    "Actual vs Predicted Values (Unnormalized)"
)

legend = chart1.add_legend().set_dragging_mode("draggable")

# Add line series for actual and predicted values
actual_series = chart1.add_line_series().set_name("Actual Values")
actual_series.set_line_color(lc.Color(255, 255, 0)).set_line_thickness(2)

predicted_series = chart1.add_line_series().set_name("Predicted Values")
predicted_series.set_line_color(lc.Color(0, 128, 255)).set_line_thickness(2)

legend.add(actual_series).add(predicted_series)
# ---- Second Row: Heatmap with Predictions ----
chart2 = dashboard.ChartXY(row_index=1, column_index=0).set_title(
    "Certainty Heatmap Streaming with Line Chart"
)


# Add a heatmap grid series
heatmap_series = chart2.add_heatmap_grid_series(
    columns=columns, rows=num_rows, data_order="rows"
).set_name("Real-Time Certainty Heatmap")
heatmap_series.set_step(
    x=(len(y_pred_shifted) - 1) / columns,  # Match the time step
    y=(y_max - y_min) / num_rows,  # Scale y step to span the prediction range
)

# Set the color palette for the heatmap
heatmap_series.set_palette_coloring(
    steps=[
        {
            "value": 0.0,
            "color": lc.Color(0, 0, 0, 128),
        },  # Transparent for low certainty
        {
            "value": 0.25,
            "color": lc.Color(255, 255, 0, 128),
        },  # Yellow for medium certainty
        {
            "value": 0.5,
            "color": lc.Color(255, 192, 0, 255),
        },  # Orange for higher certainty
        {"value": 1.0, "color": lc.Color(255, 0, 0, 255)},  # Red for maximum certainty
    ],
    look_up_property="value",
    percentage_values=True,
    interpolate=True,
)

# Customize heatmap appearance
heatmap_series.hide_wireframe()
heatmap_series.set_intensity_interpolation(True)

# Add actual predictions as a line series
prediction_series = chart2.add_line_series()
prediction_series.set_name("Predictions").set_line_color(
    lc.Color(0, 128, 255)
).set_line_thickness(2)

# Open the chart in live mode
dashboard.open(live=True, method="browser")

# Stream certainty values and predictions in batches
for t in range(0, columns, batch_size):
    for i in range(batch_size):
        if t + i < len(y_pred):
            actual_series.add(x=t + i, y=float(y_test.iloc[t + i]))
            predicted_series.add(x=t + i, y=float(y_pred[t + i]))
    # Update heatmap
    for i in range(batch_size):
        if t + i < columns:
            mat[:, t + i] = generate_certainty_heatmap(
                y_pred_shifted[t + i], certainty_values[t + i]
            )

    # Invalidate the heatmap for the updated batch
    heatmap_series.invalidate_intensity_values(mat)

    # Delay the line chart update by 2 seconds (converts delay to steps)
    time.sleep(time_delay)
    if t >= int(line_chart_delay / time_delay):
        for i in range(batch_size):
            if t + i - int(line_chart_delay / time_delay) < columns:
                prediction_series.add(
                    x=t + i - int(line_chart_delay / time_delay),
                    y=float(y_pred_shifted[t + i - int(line_chart_delay / time_delay)]),
                )
# Add remaining line chart points
for t in range(columns - int(line_chart_delay / time_delay), columns):
    prediction_series.add(x=t, y=float(y_pred_shifted[t]))
    time.sleep(0.05)


dashboard.close()
